# Single Plan Execution

In [ ]:
# Uncomment to install/upgrade ras-commander from pip
#!pip install --upgrade ras-commander

#Import the ras-commander package
from ras_commander import *

# For Development Mode, add the parent directory to the Python path
import os
import sys
from pathlib import Path

current_file = Path(os.getcwd()).resolve()
rascmdr_directory = current_file.parent

# Use insert(0) instead of append() to give highest priority to local version
if str(rascmdr_directory) not in sys.path:
    sys.path.insert(0, str(rascmdr_directory))
    
print("Loading ras-commander from local dev copy")
from ras_commander import *

##### Optional Code Cell For Development/Testing Mode (Local Copy)
##### Uncomment and run this cell instead of the pip cell above

```python
# For Development Mode, add the parent directory to the Python path
import os
import sys
from pathlib import Path

current_file = Path(os.getcwd()).resolve()
rascmdr_directory = current_file.parent

# Use insert(0) instead of append() to give highest priority to local version
if str(rascmdr_directory) not in sys.path:
    sys.path.insert(0, str(rascmdr_directory))

print("Loading ras-commander from local dev copy")
from ras_commander import *
```

In [3]:
import os
import sys
from pathlib import Path
import numpy as np
import pandas as pd
from IPython import display
import matplotlib.pyplot as plt
import psutil  # For getting system CPU info
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import subprocess
import shutil

## Setting Up Our Working Environment

Let's set up our working directory and paths to example projects. We'll also check the number of available CPU cores on this system.

In [4]:
# Extract the Bald Eagle Creek example project
# The extract_project method downloads the project from GitHub if not already present,
# and extracts it to the example_projects folder
bald_eagle_path = RasExamples.extract_project("Balde Eagle Creek", output_path="example_projects_05_single_plan_execution")
print(f"Extracted project to: {bald_eagle_path}")  


# Verify the path exists
print(f"Bald Eagle Creek project exists: {bald_eagle_path.exists()}")

2025-12-02 16:52:37 - ras_commander.RasExamples - INFO - Found zip file: c:\GH\ras-commander\examples\Example_Projects_6_6.zip
2025-12-02 16:52:37 - ras_commander.RasExamples - INFO - Loading project data from CSV...
2025-12-02 16:52:37 - ras_commander.RasExamples - INFO - Loaded 68 projects from CSV.
2025-12-02 16:52:37 - ras_commander.RasExamples - INFO - ----- RasExamples Extracting Project -----
2025-12-02 16:52:37 - ras_commander.RasExamples - INFO - Extracting project 'Balde Eagle Creek'
2025-12-02 16:52:37 - ras_commander.RasExamples - INFO - Project 'Balde Eagle Creek' already exists. Deleting existing folder...
2025-12-02 16:52:37 - ras_commander.RasExamples - INFO - Existing folder for project 'Balde Eagle Creek' has been deleted.
2025-12-02 16:52:37 - ras_commander.RasExamples - INFO - Successfully extracted project 'Balde Eagle Creek' to c:\GH\ras-commander\examples\example_projects\Balde Eagle Creek


Extracted project to: c:\GH\ras-commander\examples\example_projects\Balde Eagle Creek
Bald Eagle Creek project exists: True


In [5]:
# Define paths to example projects
examples_dir = bald_eagle_path.parent

# Define computation output paths
compute_dest_folder = examples_dir / "compute_test"

# Check system resources
cpu_count = psutil.cpu_count(logical=True)
physical_cpu_count = psutil.cpu_count(logical=False)
print(f"System has {physical_cpu_count} physical CPU cores ({cpu_count} logical cores)")
print(f"For HEC-RAS computation, it's often most efficient to use 2-8 cores")

System has 8 physical CPU cores (8 logical cores)
For HEC-RAS computation, it's often most efficient to use 2-8 cores


## Understanding the RasCmdr.compute_plan Method

Before we dive into execution, let's understand the `compute_plan` method from the `RasCmdr` class, which is the core function for running HEC-RAS simulations.

### Key Parameters

- `plan_number` (str, Path): The plan number to execute or the full path to the plan file
- `dest_folder` (str, Path, optional): Destination folder for computation
- `ras_object` (RasPrj, optional): Specific RAS object to use (defaults to global `ras`)
- `clear_geompre` (bool, optional): Whether to clear geometry preprocessor files (default: False)
- `num_cores` (int, optional): Number of processor cores to use (default: None, uses plan settings)
- `overwrite_dest` (bool, optional): Whether to overwrite the destination folder if it exists (default: False)

### Returns
- `bool`: True if the execution was successful, False otherwise

### Key Concepts

1. **Destination Folder**: By default, the simulation runs in the original project folder. Specifying a destination folder creates a copy of the project in that location for execution, leaving the original project untouched.

2. **Number of Cores**: HEC-RAS can use multiple processor cores to speed up computation. The optimal number depends on the model complexity and your computer's specifications. Generally:
   - 1-2 cores: Good for small models, highest efficiency per core
   - 3-8 cores: Good balance for most models
   - >8 cores: Diminishing returns, may actually be slower due to overhead

3. **Geometry Preprocessor Files**: These files store precomputed hydraulic properties. Clearing them forces HEC-RAS to recompute these properties, which is useful after making geometry changes.

4. **Overwrite Destination**: Controls whether an existing destination folder should be overwritten. This is a safety feature to prevent accidental deletion of important results.

## Step 1: Project Initialization

Let's initialize the HEC-RAS project using the `init_ras_project()` function.

In [6]:
# Initialize the HEC-RAS project
init_ras_project(bald_eagle_path, "6.6")
print(f"Initialized HEC-RAS project: {ras.project_name}")

2025-12-02 16:52:37 - ras_commander.RasMap - INFO - Successfully parsed RASMapper file: C:\GH\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.rasmap


Initialized HEC-RAS project: BaldEagle


## Step 2: Explore Available Plans

Let's examine the available plans in the project to understand what we're working with.

In [7]:
# Display the available plans in the project
print("Available plans in the project:")
display.display(ras.plan_df)

# Let's check the current setting for number of cores in the plans
print("\nCurrent core settings for plans:")
for plan_num in ras.plan_df['plan_number']:
    # Check all three core parameters
    d1_cores = RasPlan.get_plan_value(plan_num, "UNET D1 Cores")
    d2_cores = RasPlan.get_plan_value(plan_num, "UNET D2 Cores") 
    ps_cores = RasPlan.get_plan_value(plan_num, "PS Cores")
    
    print(f"Plan {plan_num}'s Existing Settings:")
    print(f"  1D Cores: {d1_cores}")
    print(f"  2D Cores: {d2_cores}")
    print(f"  Pump Station Cores: {ps_cores}")

Available plans in the project:


,plan_number,unsteady_number,geometry_number,Plan Title,Program Version,Short Identifier,Simulation Date,Computation Interval,Mapping Interval,Run HTab,...,PS Cores,DSS File,Friction Slope Method,HDF_Results_Path,Geom File,Geom Path,Flow File,Flow Path,full_path,flow_type
0,01,02,01,Unsteady with Bridges and Dam,5.00,UnsteadyFlow,"18FEB1999,0000,24FEB1999,0500",2MIN,1HOUR,1,...,None,dss,2,None,01,C:\GH\ras-commander\examples\example_projects\...,02,C:\GH\ras-commander\examples\example_projects\...,C:\GH\ras-commander\examples\example_projects\...,Unsteady
1,02,None,01,Steady Flow Run,NaN,SteadyRun,"02/18/1999,0000,02/24/1999,0500",2MIN,NaN,1,...,None,dss,1,None,01,C:\GH\ras-commander\examples\example_projects\...,02,C:\GH\ras-commander\examples\example_projects\...,C:\GH\ras-commander\examples\example_projects\...,Steady


2025-12-02 16:52:37 - ras_commander.RasPlan - ERROR - Key 'PS Cores' not found in the plan file.
2025-12-02 16:52:37 - ras_commander.RasPlan - ERROR - Key 'UNET D1 Cores' not found in the plan file.
2025-12-02 16:52:37 - ras_commander.RasPlan - ERROR - Key 'UNET D2 Cores' not found in the plan file.
2025-12-02 16:52:37 - ras_commander.RasPlan - ERROR - Key 'PS Cores' not found in the plan file.



Current core settings for plans:
Plan 01's Existing Settings:
  1D Cores: 0
  2D Cores: 0
  Pump Station Cores: None
Plan 02's Existing Settings:
  1D Cores: None
  2D Cores: None
  Pump Station Cores: None


## Step 3: Create a Destination Folder Structure

Now, let's prepare a destination folder for our computation. This allows us to run simulations without modifying the original project files.

In [8]:
# Create a destination folder path
dest_folder = examples_dir / "compute_test_cores"

# Check if the destination folder already exists
if dest_folder.exists():
    print(f"Destination folder already exists: {dest_folder}")
    print("We'll use overwrite_dest=True to replace it")
else:
    print(f"Destination folder will be created: {dest_folder}")

Destination folder will be created: c:\GH\ras-commander\examples\example_projects\compute_test_cores


## Step 4: Execute a Plan with a Specified Number of Cores

Now we're ready to execute a plan with a specified number of cores, overwriting the destination folder if it exists. This is the core functionality demonstrated in Example 5 of the original script.

In [9]:
# Select a plan and number of cores
plan_number = "01"
num_cores = 2  # Specify the number of cores to use

print(f"Executing plan {plan_number} with {num_cores} cores...")
print(f"Destination folder: {dest_folder}")

# Record the start time
start_time = time.time()

# Execute the plan with specified parameters
success = RasCmdr.compute_plan(
    plan_number,              # The plan to execute
    dest_folder=dest_folder,  # Where to run the simulation
    num_cores=num_cores,      # Number of processor cores to use
    overwrite_dest=True       # Overwrite destination folder if it exists
)

# Record the end time and calculate duration
end_time = time.time()
duration = end_time - start_time

# Report results
if success:
    print(f"✅ Plan {plan_number} executed successfully using {num_cores} cores")
    print(f"Execution time: {duration:.2f} seconds")
else:
    print(f"❌ Plan {plan_number} execution failed")
    print(f"Time elapsed: {duration:.2f} seconds")

2025-12-02 16:52:37 - ras_commander.RasCmdr - INFO - Using ras_object with project folder: C:\GH\ras-commander\examples\example_projects\Balde Eagle Creek
2025-12-02 16:52:37 - ras_commander.RasCmdr - INFO - Copied project folder to destination: c:\GH\ras-commander\examples\example_projects\compute_test_cores
2025-12-02 16:52:37 - ras_commander.RasMap - INFO - Successfully parsed RASMapper file: c:\GH\ras-commander\examples\example_projects\compute_test_cores\BaldEagle.rasmap
2025-12-02 16:52:37 - ras_commander.RasUtils - INFO - Using provided plan file path: c:\GH\ras-commander\examples\example_projects\compute_test_cores\BaldEagle.p01
2025-12-02 16:52:37 - ras_commander.RasUtils - INFO - Successfully updated file: c:\GH\ras-commander\examples\example_projects\compute_test_cores\BaldEagle.p01
2025-12-02 16:52:37 - ras_commander.RasCmdr - INFO - Set number of cores to 2 for plan: 01
2025-12-02 16:52:37 - ras_commander.RasCmdr - INFO - Running HEC-RAS from the Command Line:
2025-12-02 1

Executing plan 01 with 2 cores...
Destination folder: c:\GH\ras-commander\examples\example_projects\compute_test_cores


2025-12-02 16:54:09 - ras_commander.RasCmdr - INFO - HEC-RAS execution completed for plan: 01
2025-12-02 16:54:09 - ras_commander.RasCmdr - INFO - Total run time for plan 01: 91.70 seconds


✅ Plan 01 executed successfully using 2 cores
Execution time: 91.79 seconds


## Step 5: Verify Results

After execution, let's verify the results by checking the results paths and examining the destination folder.

In [10]:
# Verify that the destination folder exists and contains the expected files
if dest_folder.exists():
    print(f"Destination folder exists: {dest_folder}")
    
    # List the key files in the destination folder
    print("\nKey files in destination folder:")
    project_files = list(dest_folder.glob(f"{ras.project_name}.*"))
    for file in project_files[:10]:  # Show first 10 files
        file_size = file.stat().st_size / 1024  # Size in KB
        print(f"  {file.name}: {file_size:.1f} KB")
    
    if len(project_files) > 10:
        print(f"  ... and {len(project_files) - 10} more files")
    
    # Check for HDF result files
    print("\nHDF result files:")
    hdf_files = list(dest_folder.glob(f"*.hdf"))
    for file in hdf_files:
        file_size = file.stat().st_size / (1024 * 1024)  # Size in MB
        print(f"  {file.name}: {file_size:.1f} MB")
else:
    print(f"Destination folder does not exist: {dest_folder}")

Destination folder exists: c:\GH\ras-commander\examples\example_projects\compute_test_cores

Key files in destination folder:
  BaldEagle.b01: 9.3 KB
  BaldEagle.bco01: 2.2 KB
  BaldEagle.c01: 522.1 KB
  BaldEagle.dss: 3601.5 KB
  BaldEagle.f01: 1209.0 KB
  BaldEagle.f02: 1.5 KB
  BaldEagle.g01: 513.6 KB
  BaldEagle.g01.gmz: 372.6 KB
  BaldEagle.g01.hdf: 3920.5 KB
  BaldEagle.gis: 127.8 KB
  ... and 13 more files

HDF result files:
  BaldEagle.g01.hdf: 3.8 MB
  BaldEagle.p01.hdf: 7.4 MB
  BaldEagle.u02.hdf: 0.0 MB


In [11]:
# Since we are now working in the dest_folder, init_ras_project in that folder 

init_ras_project(dest_folder)

2025-12-02 16:54:09 - ras_commander.RasPrj - INFO - No HEC-RAS Version Specified.Attempting to detect HEC-RAS version from plan files.
2025-12-02 16:54:09 - ras_commander.RasPrj - INFO - Searching for plan files in C:\GH\ras-commander\examples\example_projects\compute_test_cores
2025-12-02 16:54:09 - ras_commander.RasPrj - INFO - Found plan file: BaldEagle.p01
2025-12-02 16:54:09 - ras_commander.RasPrj - INFO - Successfully read plan file with utf-8 encoding
2025-12-02 16:54:09 - ras_commander.RasPrj - INFO - Found Program Version=5.00 in BaldEagle.p01
2025-12-02 16:54:09 - ras_commander.RasPrj - INFO - Checking RAS executable path: C:\Program Files (x86)\HEC\HEC-RAS\5.0\Ras.exe
2025-12-02 16:54:09 - ras_commander.RasMap - INFO - Successfully parsed RASMapper file: C:\GH\ras-commander\examples\example_projects\compute_test_cores\BaldEagle.rasmap


## Step 6: Extract Computation Messages

After successful plan execution, we can extract detailed computation messages that provide insights into:
- Computation time and performance metrics
- Warning messages and errors (if any)
- Convergence information
- Process timing breakdown

The `HdfResultsPlan.get_compute_messages()` function automatically extracts messages from the HDF file, with fallback to .txt files for older HEC-RAS versions.

In [13]:
# Extract and display computation messages
from ras_commander import HdfResultsPlan

print("="*80)
print("EXTRACTING COMPUTATION MESSAGES")
print("="*80)

# Extract messages using plan number
compute_msgs = HdfResultsPlan.get_compute_messages(plan_number)

if compute_msgs:
    print(f"\nSuccessfully extracted computation messages ({len(compute_msgs)} characters)\n")
    
    # Display first 1000 characters
    print("First 1000 characters of computation messages:")
    print("-" * 80)
    print(compute_msgs[:1000])
    
    if len(compute_msgs) > 1000:
        print("\n... (message truncated for display) ...")
        print(f"\nTotal message length: {len(compute_msgs)} characters")
    
    # Check for warnings or errors
    print("\n" + "="*80)
    print("CHECKING FOR WARNINGS/ERRORS")
    print("="*80)
    
    lines = compute_msgs.split('\n')
    warnings_errors = [line for line in lines if 'warning' in line.lower() or 'error' in line.lower()]
    
    if warnings_errors:
        print(f"Found {len(warnings_errors)} warning/error messages:")
        for msg in warnings_errors[:10]:  # Show first 10
            print(f"  - {msg.strip()}")
    else:
        print("✓ No warnings or errors found in computation messages")
else:
    print("⚠ No computation messages available")
    print("This may indicate the plan was not computed or messages are not available.")

print("\n" + "="*80)

2025-12-02 16:57:29 - ras_commander.hdf.HdfResultsPlan - INFO - Final validated file path: C:\GH\ras-commander\examples\example_projects\compute_test_cores\BaldEagle.p01.hdf
2025-12-02 16:57:29 - ras_commander.hdf.HdfResultsPlan - INFO - Reading computation messages from HDF: BaldEagle.p01.hdf
2025-12-02 16:57:29 - ras_commander.hdf.HdfResultsPlan - INFO - Successfully extracted 1693 characters from HDF


EXTRACTING COMPUTATION MESSAGES

Successfully extracted computation messages (1693 characters)

First 1000 characters of computation messages:
--------------------------------------------------------------------------------
Plan: 'Unsteady with Bridges and Dam' (BaldEagle.p01)
Simulation started at: 02Dec2025 04:52:39 PM

Writing Plan GIS Data...
Completed Writing Plan GIS Data
Writing Geometry...
Computing Bank Lines
Bank lines generated in 108 ms
Computing Edge Lines
Edge Lines generated in 48 ms
Computing XS Interpolation Surface
XS Interpolation Surface generated in 115 ms
Completed Writing Geometry
Writing Event Conditions ...
Completed Writing Event Condition Data

	
Geometric Preprocessor HEC-RAS 6.6 September 2024
 

Finished Processing Geometry


Performing Unsteady Flow Simulation  HEC-RAS 6.6 September 2024
 
	
Unsteady Input Summary:
     1D Unsteady Finite Difference Numerical Solution

Overall Volume Accounting Error in Acre Feet:    -29.5468461514
Overall Volume Accounti

In [14]:
# Check the results path using the RasPlan.get_results_path method
# First, initialize a RAS object using the destination folder
try:
    dest_ras = RasPrj()
    init_ras_project(dest_folder, "6.6", ras_object=dest_ras)
    
    # Get the results path for the plan we just executed
    results_path = RasPlan.get_results_path(plan_number, ras_object=dest_ras)
    
    if results_path:
        print(f"Results for plan {plan_number} are located at: {results_path}")
        
        # Check if the file exists and get its size
        results_file = Path(results_path)
        if results_file.exists():
            size_mb = results_file.stat().st_size / (1024 * 1024)
            print(f"Results file size: {size_mb:.2f} MB")
    else:
        print(f"No results found for plan {plan_number} in the destination folder")
except Exception as e:
    print(f"Error checking results: {e}")

2025-12-02 16:57:29 - ras_commander.RasMap - INFO - Successfully parsed RASMapper file: C:\GH\ras-commander\examples\example_projects\compute_test_cores\BaldEagle.rasmap
2025-12-02 16:57:29 - ras_commander.RasMap - INFO - Successfully parsed RASMapper file: C:\GH\ras-commander\examples\example_projects\compute_test_cores\BaldEagle.rasmap


Results for plan 01 are located at: C:\GH\ras-commander\examples\example_projects\compute_test_cores\BaldEagle.p01.hdf
Results file size: 7.41 MB


## Summary of Single Plan Execution Options

The `RasCmdr.compute_plan()` method provides a flexible way to execute HEC-RAS plans with various options. Here's a summary of the key parameters we've explored:

1. **Basic Execution**: Simply provide a plan number
   ```python
   RasCmdr.compute_plan("01")
   ```

2. **Destination Folder**: Run in a separate folder to preserve the original project
   ```python
   RasCmdr.compute_plan("01", dest_folder="path/to/folder")
   ```

3. **Number of Cores**: Control the CPU resources used
   ```python
   RasCmdr.compute_plan("01", num_cores=2)
   ```

4. **Overwrite Destination**: Replace existing computation folders
   ```python
   RasCmdr.compute_plan("01", dest_folder="path/to/folder", overwrite_dest=True)
   ```

5. **Clear Geometry Preprocessor**: Force recalculation of geometric properties
   ```python
   RasCmdr.compute_plan("01", clear_geompre=True)
   ```

6. **Combined Options**: Use multiple options together
   ```python
   RasCmdr.compute_plan(
       "01",
       dest_folder="path/to/folder",
       num_cores=2,
       clear_geompre=True,
       overwrite_dest=True
   )
   ```

### Next Steps

To further enhance your HEC-RAS automation, consider exploring:

1. **Parallel Execution**: Use `RasCmdr.compute_parallel()` to run multiple plans simultaneously
2. **Test Mode**: Use `RasCmdr.compute_test_mode()` for testing purposes
3. **Pre-Processing**: Modify plans, geometries, and unsteady flows before execution
4. **Post-Processing**: Analyze results after computation
5. **Batch Processing**: Create scripts for parameter sweeps or scenario analysis

These advanced topics are covered in other examples and documentation for the RAS Commander library.